# Problem Statement

The objective is to minimize the cost of maintenance. X is the number of machines that need to be maintained in a certain state. Y is the number of machines in a certain state. 
M is the minimum number of machines required to be operating. Probability of deteriorating from from tate to another is defined. Maintenance cost is defined.

The code assumes 3 states only.

# Machines

In [1]:
import gurobipy as grb
from gurobipy import GRB

# Create grb model 
mod1 = grb.Model("Machines")

Academic license - for non-commercial use only


In [2]:
#Create Dictionary

C={} 

#Initiating costs
#Failure mode
C[0]=6
#Operating states
C[1]=3 #C[s]
C[2]=1


In [3]:
state = []
r=3
for i in range(r):
    state.append(i)    
print(state)
final=i
print(final)

[0, 1, 2]
2


In [4]:
num = []
n=r-1
for i in range(n):
    num.append(i)
print(num)

[0, 1]


In [5]:
day = []
dy=6
for i in range(1,dy):
    day.append(i)
print(day)

[1, 2, 3, 4, 5]


In [6]:
#Create variables (Change X and Y)
X = mod1.addVars(state,day,lb=0,vtype=GRB.INTEGER,name="X") # Number of machines maintained in state s and day d
Y = mod1.addVars(state,day,lb=0,vtype=GRB.INTEGER,name="Y") #Number of operating machines in state s and day d 

In [7]:
#Objective function (Minimization)
mod1.setObjective( grb.quicksum( X[s,d] * C[s] for s in state for d in day ), grb.GRB.MINIMIZE)

In [9]:
#Assuming only 3 states
P={}
#Initiating probabilities 
P[1,0] = 0.9 #P[s,s']
P[2,1]= 0.3
P[2,0]= 0.1


#Random values
Y[0,0] = 0
Y[1,0] = 0
Y[2,0] = 100
Y[3,0] = 0
X[0,0] = 0
X[1,0] = 0
X[2,0] = 0
X[3,0] = 0

In [10]:
mod1.update()
#Constraints

mod1.addConstrs( (X[s,d] <= Y[s,d] for s in state for d in day), name="XlessthanY")

#State constraints

mod1.addConstrs( (Y[n,d]  == (Y[n,d-1]- X[n,d-1])*(1-grb.quicksum(P[n,s] for s in range(0,n))) +grb.quicksum((Y[s,d-1]- X[s,d-1])*P[s,n]  for s in range (n+1,r)) for n in num for d in day) ,name= 'StateN')
                

mod1.addConstrs( ( Y[final,d] == Y[final,d-1]*(1-grb.quicksum(P[final,s] for s in range(0,r-1))) + grb.quicksum(X[s,d-1] for s in range(0,r-1) ) for d in day) ,name= 'FinalState')

#Minimum number of operating machines 
M = 80 #Random value
mod1.addConstrs( (  grb.quicksum(Y[s,d] for s in range(1,r))  >= M for d in day )  , name= 'minimum' )

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

In [11]:
range(0,r)

[0, 1, 2]

In [12]:
range(0,n)

[0, 1]

In [13]:
mod1.update()
mod1.optimize()

Optimize a model with 35 rows, 30 columns and 107 nonzeros
Variable types: 0 continuous, 30 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [1e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 8e+01]
Presolve removed 11 rows and 7 columns
Presolve time: 0.00s
Presolved: 24 rows, 23 columns, 82 nonzeros
Variable types: 0 continuous, 23 integer (0 binary)

Root relaxation: objective 3.464640e+02, 19 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  346.46400    0   11          -  346.46400      -     -    0s
H    0     0                     408.0000000  346.46400  15.1%     -    0s
     0     0  355.99504    0    3  408.00000  355.99504  12.7%     -    0s
H    0     0                     366.0000000  355.99504  2.73%     -    0s
     0     0  356.02878    0    5  366.00000  356.0

In [14]:
mod1.write("Machines.lp")
f = open('Machines.lp', 'r')
print (f.read())
f.close()

\ Model Machines
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  6 X[0,1] + 6 X[0,2] + 6 X[0,3] + 6 X[0,4] + 6 X[0,5] + 3 X[1,1] + 3 X[1,2]
   + 3 X[1,3] + 3 X[1,4] + 3 X[1,5] + X[2,1] + X[2,2] + X[2,3] + X[2,4]
   + X[2,5]
Subject To
 XlessthanY[0,1]: X[0,1] - Y[0,1] <= 0
 XlessthanY[0,2]: X[0,2] - Y[0,2] <= 0
 XlessthanY[0,3]: X[0,3] - Y[0,3] <= 0
 XlessthanY[0,4]: X[0,4] - Y[0,4] <= 0
 XlessthanY[0,5]: X[0,5] - Y[0,5] <= 0
 XlessthanY[1,1]: X[1,1] - Y[1,1] <= 0
 XlessthanY[1,2]: X[1,2] - Y[1,2] <= 0
 XlessthanY[1,3]: X[1,3] - Y[1,3] <= 0
 XlessthanY[1,4]: X[1,4] - Y[1,4] <= 0
 XlessthanY[1,5]: X[1,5] - Y[1,5] <= 0
 XlessthanY[2,1]: X[2,1] - Y[2,1] <= 0
 XlessthanY[2,2]: X[2,2] - Y[2,2] <= 0
 XlessthanY[2,3]: X[2,3] - Y[2,3] <= 0
 XlessthanY[2,4]: X[2,4] - Y[2,4] <= 0
 XlessthanY[2,5]: X[2,5] - Y[2,5] <= 0
 StateN[0,1]: Y[0,1] = 10
 StateN[0,2]: X[0,1] + 0.9 X[1,1] + 0.1 X[2,1] - Y[0,1] + Y[0,2]
   - 0.9 Y[1,1] - 0.1 Y[2,1] = 0
 StateN[0,3]: 

In [15]:
for v in mod1.getVars(): 
    print('%s %g' % (v.varName, v.x))
print('Obj: %g' % mod1.objVal)

X[0,1] 4
X[0,2] 0
X[0,3] 13
X[0,4] -0
X[0,5] -0
X[1,1] 30
X[1,2] 18
X[1,3] 21
X[1,4] 18
X[1,5] -0
X[2,1] -0
X[2,2] -0
X[2,3] -0
X[2,4] -0
X[2,5] -0
Y[0,1] 10
Y[0,2] 12
Y[0,3] 19
Y[0,4] 12
Y[0,5] 19
Y[1,1] 30
Y[1,2] 18
Y[1,3] 21
Y[1,4] 18
Y[1,5] 21
Y[2,1] 60
Y[2,2] 70
Y[2,3] 60
Y[2,4] 70
Y[2,5] 60
Obj: 363


In [16]:
mod1.printAttr('X')


    Variable            X 
-------------------------
      X[0,1]            4 
      X[0,3]           13 
      X[1,1]           30 
      X[1,2]           18 
      X[1,3]           21 
      X[1,4]           18 
      Y[0,1]           10 
      Y[0,2]           12 
      Y[0,3]           19 
      Y[0,4]           12 
      Y[0,5]           19 
      Y[1,1]           30 
      Y[1,2]           18 
      Y[1,3]           21 
      Y[1,4]           18 
      Y[1,5]           21 
      Y[2,1]           60 
      Y[2,2]           70 
      Y[2,3]           60 
      Y[2,4]           70 
      Y[2,5]           60 
